# KnitHAT 

This notebook is step one in the process of building [KnitHAT](http://knithat.works), which recommends hat patterns from Ravelry based on user-uploaded images. KnitHAT was my Insight Data Science fellowship project, and was built in only three weeks.

This code scrapes the IDs of only hat patterns from Ravelry.

In [42]:
## with HUGE thanks to https://github.com/yellowcrayon/RavelryPatternDataScraper

import json
import time as time

In [34]:
import requests as rq

user = [redacted]
pswd = [redacted]

response = rq.get('https://api.ravelry.com/patterns/search.json?page_size=10&page=1', auth=(user, pswd))
print(response)

<Response [200]>


In [35]:
print(response.json()['paginator'])

{'page_count': 10000, 'page': 1, 'page_size': 10, 'results': 100000, 'last_page': 10000}


In [36]:
def printPatternNames(response):
    #This function takes a Ravelry response from the search.json? endpoint and prints all the pattern names it finds
    #This function is indended to help with debugging, to ensure that the patterns you're getting back are the ones you expect
    patternData = response.json()['patterns']
    
    for i in range(len(patternData)):
        print(patternData[i]['name'])

In [37]:
def scrapePatternIDs(numPages,searchVals,user,pswd,waitTime):
    #Loop through the pages of results, then through the results on each page
    #Save the pattern ID numbers to a list
    
    IDs = [] #Initialize the list of pattern IDs
    
    for page in range(1,numPages+1): #Redo the query for the first page too so that we can append all of those ID numbers below
    #Note that page is not a python index, but instead it is the index of the results page that we're asking the API to return.
    #This results page index starts at 1 (1 is the first page) and ends at numPages (numPages is the last page), which is why we use range(1,numPages+1).
        searchVals['page'] = str(page) #Set the page # of the search results to ask for
        response = rq.get('https://api.ravelry.com/patterns/search.json?', params = searchVals, auth = (user,pswd)) #Make the query
        print(response) #Print the response code to check that our query was successful
    
        time.sleep(waitTime) #Wait for a few seconds between each request so as not to spook the API
      
        try:
            patternData = response.json()['patterns'] #This is a list containing one dictionary for each pattern result
            for pat in range(len(patternData)):
            
                try:
                    IDs.append(patternData[pat]['id']) #Append the pattern ID numbers to the IDs list
                
                except:
                    print('Problem on page',page,', pattern ',pat)
                
        except:
            print('Problem on page ',page)
        
    #Sort the list of IDs
    IDs.sort()
    print('Number of IDs found: ',len(IDs))

    #Eliminate duplicates in the IDs list by converting it to a set
    IDs = set(IDs)

    #Convert IDs set back into a list
    IDs = list(IDs)
    print('Number of unique IDs found: ',len(IDs))
    
    return IDs

In [38]:
def saveIDs(IDs,path):
    
    mode = 'w' #put this file in write mode
    
    IDstrings = map(str,IDs) #Convert the ID list into a list of strings
    IDstrings = map(lambda x: x + '\n', IDstrings) #Add a newline character to each ID string
    #Note that there will be an extra newline character at the end of the file

    #Save the list of IDs as a CSV file
    with open(path,mode) as out:
        out.writelines(map(str,IDstrings))

In [39]:
pageSize = 500 #Number of results to ask for per page
craft = 'knitting' #Get the knitted patterns only
# PatternCategory = 'Hat'
in_name = 'hat'
photo = 'yes'
waitTime = 5 #Number of seconds to wait between requests to avoid spooking the API

#100 results per page, start with page 1 of the results and sort the results by the date created
searchVals = {'page_size': str(pageSize), 'page': '1', 'craft': craft, 'query': in_name, 'photo': photo, 'sort': 'created'} #searchVals is the list of parameters to send to the API

#Make an initial request to the API to figure out how many pages of results there will be for our desired pageSize
response = rq.get('https://api.ravelry.com/patterns/search.json?', params = searchVals, auth = (user,pswd))
print(response) #Print the response code so we can check whether the query was successful
printPatternNames(response) #Print the names of the patterns found in this first call, so that we can check that we are getting the patterns we expect
numPages = response.json()['paginator']['last_page'] #Number of pages of results

#Display some info about # of requests that will be made and # of results per page
print('Making ',numPages,' requests: ',numPages,' pages of data with ',pageSize,' results per page.')

#Display how long this code should take to run
print('This code should take approximately ',waitTime*numPages/60,' minutes to complete.')

<Response [200]>
Kindness Is Cool Beanie
Hats with a Mission
Scalloped Stitches
Margretes kyse
Minions hat
The Golden Spirit of Scotland
The Newport Beanie
Sylvan Tales Hat
F940 Ribbed Lace Cowl
Punnet
Rayas Hat
Faulty Towers
Baby Turban
Garter Stitch Bonnet
bats
Skulls
Pebbles By The Sea Hat
Wave Hat Set
Double Knit Reversible Beanie
Neva Hat
Newborn foxy bonnet
Cocoon Slouchy Hat
254-13 Lang Yarns Viva
254-04 Lang Yarns Lusso
Bushel Beanie
Yazmin Newborn Bonnet
Mystère
Sagakyse
Aron's Hat
Stashbuster Hat
Emily Hat
Cabled Tassels Hat
Sunrise Hat
Striped cap with bobble
#23 Hat
Watermelon Slice Hat
Mill Cap
Sól Shield Hat
#17 Hat and Fingerless Mitts
The Spero Beanie
Hawthorn Ridge
Sunday River Hat
Teazle
Enve
Cabled Dog Hat
Mohawk Dog Hat
Chasm Hat
Branta Leucopsis
Crater Lake Beanie
Baratheon Knit Hat (Game of Thrones)
Targaryen Knit Hat
GOT Stark Knit Hat
#15 Slouchy Hat
Emma Joy Hat
Kaiju in Seattle
HueLue
Hickory Beanie
Vertical Stripe Hat
EK4 Baby Hat
Mixed Pompom Hat
Lila Bonnet

In [43]:
WaitTime = 5 #Number of seconds to wait between requests to avoid spooking the API
IDs = scrapePatternIDs(numPages,searchVals,user,pswd,waitTime)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [45]:
# save pattern IDs to text file

path = 'HatPatternIDs.txt'
saveIDs(IDs,path)